In [54]:
import pandas as pd
news_data = pd.read_csv('/content/drive/MyDrive/DataMining TeamProject/user_based_CF/clustering_news.csv', encoding = 'euc-kr')
rating_data = pd.read_csv('/content/drive/MyDrive/DataMining TeamProject/user_based_CF/weights.csv')
rating_data.head()

,userId,articleId,scrap,view,weight
0,1,232,2,1,3
1,1,124,0,1,1
2,1,92,2,3,5
3,1,405,2,1,3
4,1,851,0,1,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
news_data.head()

,articleId,title,content,content_cleand,labels
0,0,中 니오 전기차 인도물량 65%↓…반도체난 직격탄,10월 3667대 불과… 주문량은 역대 최고 기록 경신 중국 전기차업체 니오의 질주...,월 대 불과 주문량은 역대 최고 기록 경신 중국 전기차업체 니오의 질주...,8
1,1,페북도 링 위 오른다…애플에 스마트워치 도전장,점유율 1위 애플워치…메타와의 대결구도 관심 삼성전자 갤럭시워치도 메타와 유력 경쟁...,점유율 위 애플워치 메타와의 대결구도 관심 삼성전자 갤럭시워치도 메타와 유력 경쟁...,2
2,2,넥슨 ‘던전앤파이터 모바일’ 내년 1분기 국내 출시,넥슨은 1일 자회사 네오플이 개발한 2D 액션 역할수행게임 RPG ‘던전앤파이터 모...,넥슨은 일 자회사 네오플이 개발한 D 액션 역할수행게임 RPG 던전앤파이터 모...,20
3,3,KT “개인·기업 천 원 소상공인 7천 원 감면”…고객 분통,앵커 통신마비 사태로 전국적 혼란을 초래했던 KT가 u200b보상 방안을 내놨습니다...,앵커 통신마비 사태로 전국적 혼란을 초래했던 KT가 u b보상 방안을 내놨습니다...,28
4,4,‘가상 공장에서 물품 제조’…메타버스 체험 생생하게,KBS 대전 앵커 쌍방향 소통이 가능한 가상세계 디지털 세상의 놀이터 라고 불리는 ...,KBS 대전 앵커 쌍방향 소통이 가능한 가상세계 디지털 세상의 놀이터 라고 불리는 ...,4


In [56]:
user_news_rating = rating_data.pivot_table(index = 'userId', columns = 'articleId', values = 'weight').fillna(0)
user_news_rating.head()

articleId,1,2,3,4,5,6,7,8,9,10,11,12,14,15,19,22,25,29,32,36,48,54,62,63,65,77,83,87,92,97,98,101,124,153,169,192,209,213,218,232,...,306,320,345,356,362,394,398,405,411,452,472,511,526,548,551,556,564,567,594,651,678,752,780,784,786,845,851,853,983,1452,2364,3589,3748,10929,17283,20938,33333,34895,40000,41898
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
user_news_matrix = user_news_rating.values

In [58]:
import numpy as np
user_mean = np.mean(user_news_matrix, axis=1)
user_mean.shape

(15,)

In [59]:
user_news_matrix_update = user_news_matrix - user_mean.reshape(-1,1)

In [60]:
pd.DataFrame(user_news_matrix_update, columns=user_news_rating.columns).head()

articleId,1,2,3,4,5,6,7,8,9,10,11,12,14,15,19,22,25,29,32,36,48,54,62,63,65,77,83,87,92,97,98,101,124,153,169,192,209,213,218,232,...,306,320,345,356,362,394,398,405,411,452,472,511,526,548,551,556,564,567,594,651,678,752,780,784,786,845,851,853,983,1452,2364,3589,3748,10929,17283,20938,33333,34895,40000,41898
0,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,4.845238,-0.154762,-0.154762,-0.154762,0.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,2.845238,...,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,2.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,0.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762
1,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,...,3.880952,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,0.880952,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,0.880952,0.880952,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048,-0.119048
2,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,0.904762,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,0.904762,-0.095238,0.904762,-0.095238,-0.095238,-0.095238,...,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,1.904762,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,2.904762,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238,-0.095238
3,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,2.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,...,-0.154762,0.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,3.845238,-0.154762,-0.154762,-0.154762,3.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,0.845238,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762,-0.154762
4,0.583333,-0.416667,2.583333,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,2.583333,-0.416667,-0.416667,-0.416667,0.583333,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,2.583333,0.583333,-0.416667,-0.416667,-0.416667,-0.416667,-0.416667,2.583333,-0.416667,-0.416667,-0.416667,2.583333,-0.416667,-0.416667,-0.416667,-0.416667,...,-0.416667,-0.416667,-0.416667,-0.416667,0.583333,0.583333,-0.416667,-0.416667,3.58333

In [61]:
from scipy.sparse.linalg import svds, eigs
U, S, Vt = svds(user_news_matrix_update, k=9)
S = np.diag(S)

In [62]:
svd_user_predicted_ratings = np.dot(np.dot(U,S),Vt) + user_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns=user_news_rating.columns)
df_svd_preds.head()

articleId,1,2,3,4,5,6,7,8,9,10,11,12,14,15,19,22,25,29,32,36,48,54,62,63,65,77,83,87,92,97,98,101,124,153,169,192,209,213,218,232,...,306,320,345,356,362,394,398,405,411,452,472,511,526,548,551,556,564,567,594,651,678,752,780,784,786,845,851,853,983,1452,2364,3589,3748,10929,17283,20938,33333,34895,40000,41898
0,-0.059335,0.001379,-0.074593,0.081007,0.081007,-0.116697,0.027922,0.054464,0.054464,-0.076057,0.081007,-0.062162,0.027922,0.027922,0.005034,-0.076070,0.056161,0.099944,-0.202051,0.099343,0.050182,-0.162702,0.017466,0.009903,0.005034,0.073378,0.069769,-0.021995,4.904507,0.028468,-0.011073,0.005034,1.066941,0.056161,0.069769,0.005034,0.069769,-0.167879,0.099944,2.985724,...,-0.155122,0.064368,0.091137,0.015740,0.073378,0.073378,0.041882,2.985724,-0.029137,-0.029137,-0.072618,0.031989,0.068009,-0.098004,0.041882,0.041882,-0.046616,0.099944,-0.065176,0.005034,-0.046616,-0.005791,-0.065176,-0.088233,0.068009,0.015740,1.066941,0.005034,0.064368,0.050182,-0.088233,0.042288,-0.064553,0.050182,0.050182,-0.064553,-0.007185,0.009903,-0.007185,-0.064553
1,-0.095091,-0.000085,-0.116415,0.126431,0.126431,-0.194246,0.042087,0.084259,0.084259,-0.083170,0.126431,-0.123346,0.042087,0.042087,0.010101,-0.183391,0.084264,0.169080,-0.412223,0.168826,0.076111,-0.327695,0.003171,0.120372,0.010101,0.115769,0.155620,-0.041380,-0.159735,0.040333,-0.023803,0.010101,0.102936,0.084264,0.155620,0.010101,0.155620,-0.359389,0.169080,-0.028400,...,3.721194,0.098609,0.169049,-0.007394,0.115769,0.115769,1.056751,-0.028400,-0.042733,-0.042733,-0.162262,0.142637,0.104468,-0.168753,1.056751,1.056751,-0.084414,0.169080,-0.111375,0.010101,-0.084414,0.129653,-0.111375,-0.082541,0.104468,-0.007394,0.102936,0.010101,0.098609,0.076111,-0.082541,0.084889,-0.108874,0.076111,0.076111,-0.108874,-0.016382,0.120372,-0.016382,-0.108874
2,-0.130533,-0.036404,-0.131924,0.103746,0.103746,-0.049955,0.010313,0.057030,0.057030,0.127474,0.103746,0.061878,0.010313,0.010313,0.008226,0.256170,0.114126,0.154791,0.261610,0.155084,0.094479,0.305158,0.202028,0.203167,0.008226,0.103050,0.170836,0.034632,-0.038438,0.088982,0.058242,0.008226,0.112683,0.114126,0.170836,0.008226,0.170836,0.309023,0.154791,0.037123,...,0.098529,0.111853,0.159706,0.203316,0.103050,0.103050,0.137480,0.037123,-0.039187,-0.039187,0.253593,0.191210,0.119723,0.005116,0.137480,0.137480,0.041453,0.154791,-0.003978,0.008226,0.041453,0.211583,-0.003978,0.256129,0.119723,0.203316,0.112683,0.008226,0.111853,0.094479,0.256129,0.185685,-0.017489,0.094479,0.094479,-0.017489,0.038495,0.203167,0.038495,-0.017489
3,-0.062979,0.001472,-0.079160,0.086012,0.086012,-0.124275,0.029652,0.057832,0.057832,-0.080418,0.086012,-0.066677,0.029652,0.029652,0.005380,-0.082425,0.059510,0.106705,-0.217004,0.106079,0.053199,-0.174793,0.017864,0.013518,0.005380,0.077922,0.075582,2.976425,-0.101714,0.030078,-0.011980,0.005380,0.071011,0.059510,0.075582,0.005380,0.075582,-0.180734,0.106705,-0.015351,...,-0.165786,1.068270,0.097965,0.015884,0.077922,0.077922,0.044198,-0.015351,-0.030890,-0.030890,-0.078464,0.036972,0.072135,-0.104536,0.044198,0.044198,-0.049854,0.106705,3.930503,0.005380,-0.049854,-0.001638,3.930503,-0.093182,0.072135,0.015884,0.071011,0.005380,1.068270,0.053199,-0.093182,0.045068,-0.068788,0.053199,0.053199,-0.068788,-0.007794,0.013518,-0.007794,-0.068788
4,0.952622,0.004620,2.936592,0.070593,0.070593,-0.096293,0.026611,0.048602,0.048602,-0.103929,0.070593,-0.039586,0.026611,0.026611,3.002565,-0.030753,0.049840,0.079351,0.877573,0.078870,0.044200,-0.106006,0.026387,-0.021779,3.002565,1.062577,0.045171,-0.016229,-0.078275,0.025829,-0.007548,3.002565,0.058412,0.049840,0.045171,3.002565,0.045171,-0.092421,0.079351,-0.009931,...,-0.118752,0.056313,0.065156,0.030916,1.062577,1.062577,0.039750,-0.009931,3.972558,3.972558,-0.039809,-0.002241,0.059207,-0.078392,0.039750,0.039750,-0.035648,0.079351,-0.052499,3.002565,-0.035648,-0.049654,-0.052499,-0.136212,0.059207,0.030916,0.058412,3.002565,0.056313,0.044200,-0.136212,0.016318,-0.0525

In [63]:
def recommend_news(preds_df, userID, news_df, original_ratings_df, num_recommendations):
    user_row_number = userID - 1
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) 

    news_data['articleId'] = news_data['articleId'].astype(int)
    original_ratings_df['articleId'] = original_ratings_df['articleId'].astype(int)

    user_data = original_ratings_df[original_ratings_df.userId == (userID)]

    user_full = (user_data.merge(news_df, how = 'left', left_on = 'articleId', right_on = 'articleId').
                     sort_values(['weight'], ascending=False)
                 )

    recommendations = (news_df[~news_df['articleId'].isin(user_full['articleId'])]).merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'articleId',
               right_on = 'articleId').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1]
                      

    return user_full, recommendations


In [64]:
already_rated, predictions = recommend_news(df_svd_preds, 15, news_data, rating_data, 10)
already_rated.head(10)

,userId,articleId,scrap,view,weight,title,content,content_cleand,labels
1,15,273,2,1,3,엔픽셀 서류 전형 없이 신입사원 공개채용 진행,엔픽셀 공동대표 배봉건 정현호 은 자사 첫 번째 신입사원 공개채용을 실시한다고 1일...,엔픽셀 공동대표 배봉건 정현호 은 자사 첫 번째 신입사원 공개채용을 실시한다고 일...,3
3,15,3748,2,1,3,누리호 궤도 진입 실패 원인은 3단 산화제탱크 압력 저하 때문,과기부 항우연 누리호 발사조사위원회 첫 회의 사진공동취재단 성공을 목전에 두고 아쉽...,과기부 항우연 누리호 발사조사위원회 첫 회의 사진공동취재단 성공을 목전에 두고 아쉽...,2
6,15,41898,2,1,3,홍천 어린이 희망날개 어울림 카누교실 개강,홍천 어린이 희망날개 어울림 카누교실 개강 ◇강원일보 복지법인 함께사는 강원세상 이...,홍천 어린이 희망날개 어울림 카누교실 개강 강원일보 복지법인 함께사는 강원세상 이...,3
7,15,20938,2,1,3,공급망 관련 글로벌 정상회의 발언하는 문 대통령,로마 연합뉴스 최재구 기자 문재인 대통령이 31일 현지시간 이탈리아 로마 누볼라 컨...,로마 연합뉴스 최재구 기자 문재인 대통령이 일 현지시간 이탈리아 로마 누볼라 컨...,27
0,15,40000,0,2,2,목포대연구센터 LNG 운반선 1차 방벽 소재개발 위한 업무협약,국립목포대학교 전경. 국립목포대 극저온단열시스템연구센터 이하 연구센터 는 최근 현대...,국립목포대학교 전경 국립목포대 극저온단열시스템연구센터 이하 연구센터 는 최근 현대...,3
9,15,33333,0,2,2,포즈 취하는 작곡가 박영희,서울 연합뉴스 이재희 기자 재독 작곡가 박영희가 7일 서울 중구 시청 인근에서 열린...,서울 연합뉴스 이재희 기자 재독 작곡가 박영희가 일 서울 중구 시청 인근에서 열린...,17
2,15,1452,0,1,1,포토제2회 소셜D·N·A혁신상 시상식 혁신상 받는 김선우 대표,AI 위드 휴먼 AI With Human 을 주제로 아이포럼 2021 이 2일 서울...,AI 위드 휴먼 AI With Human 을 주제로 아이포럼 이 일 서울...,4
4,15,10929,0,1,1,화성 외국인보호소 인권침해 새우꺾기 있었다,서울 뉴시스 조수정 기자 법무부 인권국장 직무대리 이상갑 법무실장이 1일 오후 서울...,서울 뉴시스 조수정 기자 법무부 인권국장 직무대리 이상갑 법무실장이 일 오후 서울...,1
5,15,48,0,1,1,기본법·특별법 난립 그만…부처별 중복 조정기능 줄여야,김승수 의원실 정부 조직개편 토론회서 큰 흐름 필요성 제시 “소사이어티 5.0 인더...,김승수 의원실 정부 조직개편 토론회서 큰 흐름 필요성 제시 소사이어티 인더...,2
8,15,17283,0,1,1,이준석 “거간꾼 해당 행위” vs 안철수 “평론가 버릇”,안철수 국민의당 대표의 야권 단일화 여부에 대한 관심이 높아지고 있는데 이준석 국민...,안철수 국민의당 대표의 야권 단일화 여부에 대한 관심이 높아지고 있는데 이준석 국민...,9


In [65]:
predictions.head(10)

,articleId,title,content,content_cleand,labels
217,218,건설자동화 시범사업 개막식 인사말 하는 노형욱,세종 뉴스1 장수영 기자 노형욱 국토교통부 장관이 1일 세종시 5 1생활권 스마트시...,세종 뉴스 장수영 기자 노형욱 국토교통부 장관이 일 세종시 생활권 스마트시...,17
29,29,소프트웨어·인공지능 교육 체험 행사 온라인 개최,다양한 소프트웨어·인공지능 교육을 체험해 볼 수 있는 온라인 행사가 열렸습니다. 과...,다양한 소프트웨어 인공지능 교육을 체험해 볼 수 있는 온라인 행사가 열렸습니다 과...,4
565,567,GC녹십자 3분기 역대 최대 매출… 영업이익 41% 증가,GC녹십자 GC녹십자가 3분기에 역대 최대 매출을 기록했다. GC녹십자는 1일 공시...,GC녹십자 GC녹십자가 분기에 역대 최대 매출을 기록했다 GC녹십자는 일 공시...,6
36,36,이동통신유통협회 “KB리브엠 혁신금융서비스 재인가 취소해야”,디지털데일리 권하영 기자 전국이동통신유통협회 KMDA 는 1일 KB국민은행 알뜰폰 ...,디지털데일리 권하영 기자 전국이동통신유통협회 KMDA 는 일 KB국민은행 알뜰폰 ...,2
343,345,자유롭게 앉아 일한다…야놀자 좌석공유제 시행,디지털데일리 임재현기자 야놀자가 사무실 근무 직원에게도 효율적인 환경을 제공할 수 ...,디지털데일리 임재현기자 야놀자가 사무실 근무 직원에게도 효율적인 환경을 제공할 수 ...,3
11,11,‘가상 공장에서 물품 제조’…메타버스 체험 생생하게,KBS 대전 앵커 쌍방향 소통이 가능한 가상세계 디지털 세상의 놀이터 라고 불리는 ...,KBS 대전 앵커 쌍방향 소통이 가능한 가상세계 디지털 세상의 놀이터 라고 불리는 ...,4
4,4,‘가상 공장에서 물품 제조’…메타버스 체험 생생하게,KBS 대전 앵커 쌍방향 소통이 가능한 가상세계 디지털 세상의 놀이터 라고 불리는 ...,KBS 대전 앵커 쌍방향 소통이 가능한 가상세계 디지털 세상의 놀이터 라고 불리는 ...,4
5,5,“점심 장사 망쳤는데…” KT 먹통 사태 70008000원 찔끔 보상,KT ‘먹통사태’ 보상안 발표 경향신문 YHAP 고개 숙여도… KT 임원들이 1일 ...,KT 먹통사태 보상안 발표 경향신문 YHAP 고개 숙여도 KT 임원들이 일 ...,28
76,77,“우리한테만 지나치게 가혹” 전기자전거 확대에 킥보드 ‘울분’,서울시는 공유킥보드 불법 주·정차 단속 및 즉시 견인 조치를 시행했다. 앞서 면허소...,서울시는 공유킥보드 불법 주 정차 단속 및 즉시 견인 조치를 시행했다 앞서 면허소...,2
392,394,KT 인재 다시는 없다…관리자 승인·테스트베드 구축,10월 초 DNS 공격 있어 디도스로 선추정… 10여년 동안 이런 사고 처음 KT가...,월 초 DNS 공격 있어 디도스로 선추정 여년 동안 이런 사고 처음 KT가...,28
